A template of the RL training, with H-network trained along side the PPO agent

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from utils import print_log

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load the created dataset
dataset_folder_path = Path("dataset", "20250707_downsampled_1min", "split")

In [4]:
# copied from 03_data_split.ipynb

# Helper functions for the new split folder structure
def load_split_data_from_folder(split_folder, split_type='train'):
    """Load aggregate data from split folder"""
    segments = []
    with open(split_folder / f'{split_type}_segments.txt', 'r') as f:
        for line in f:
            start_str, end_str = line.strip().split(' - ')
            start = datetime.fromisoformat(start_str)
            end = datetime.fromisoformat(end_str)
            segments.append((start, end))
    
    df = pd.read_pickle(split_folder / f'{split_type}_aggregate_df.pkl')
    return segments, df

def load_signatures_from_split_folder(split_folder, split_type, appliance):
    """Load appliance signatures from split folder"""
    sig_path = split_folder / 'load_signature_library' / split_type / appliance / 'load_signatures.pkl'
    ranges_path = split_folder / 'load_signature_library' / split_type / appliance / 'selected_ranges.txt'
    
    if not sig_path.exists():
        return pd.DataFrame(), []
    
    signatures_df = pd.read_pickle(sig_path)
    ranges = []
    if ranges_path.exists():
        with open(ranges_path, 'r') as f:
            for line in f:
                start, end = map(int, line.strip().split(','))
                ranges.append((start, end))
    
    return signatures_df, ranges

In [5]:
# convert datetime objects to timezone-naive datetime objects
def convert_to_naive_datetimes_df(df):
    """Convert datetime objects in DataFrame to timezone-naive datetime objects"""
    df['datetime'] = df['datetime'].apply(lambda x: x.replace(tzinfo=None) if isinstance(x, datetime) else x)

    return df

def convert_to_naive_datetimes(segments):
    """Convert datetime objects in segments to timezone-naive datetime objects"""
    return [(start.replace(tzinfo=None), end.replace(tzinfo=None)) for start, end in segments]

In [6]:
aggregate_load_segments_train, aggregate_load_df_train = load_split_data_from_folder(dataset_folder_path, 'train')
aggregate_load_segments_test, aggregate_load_df_test = load_split_data_from_folder(dataset_folder_path, 'test') 
aggregate_load_segments_validation, aggregate_load_df_validation = load_split_data_from_folder(dataset_folder_path, 'val')

In [7]:
aggregate_load_segments_train, aggregate_load_df_train = convert_to_naive_datetimes(aggregate_load_segments_train), convert_to_naive_datetimes_df(aggregate_load_df_train)
aggregate_load_segments_test, aggregate_load_df_test = convert_to_naive_datetimes(aggregate_load_segments_test), convert_to_naive_datetimes_df(aggregate_load_df_test)
aggregate_load_segments_validation, aggregate_load_df_validation = convert_to_naive_datetimes(aggregate_load_segments_validation), convert_to_naive_datetimes_df(aggregate_load_df_validation)

In [8]:
aggregate_load_df_train

,datetime,aggregate,timestamp
0,2013-01-08 00:00:05,234.000000,1357603205
1,2013-01-08 00:01:05,230.407069,1357603265
2,2013-01-08 00:02:05,230.680121,1357603325
3,2013-01-08 00:03:05,231.607379,1357603385
4,2013-01-08 00:04:05,231.280688,1357603445
...,...,...,...
231546,2013-12-30 23:55:01,176.973052,1388447701
231547,2013-12-30 23:56:01,177.850890,1388447761
231548,2013-12-30 23:57:01,177.333811,1388447821
231549,2013-12-30 23:58:01,178.462801,1388447881


In [9]:
aggregate_load_segments_train

[(datetime.datetime(2013, 1, 8, 0, 0),
  datetime.datetime(2013, 1, 8, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 1, 9, 0, 0),
  datetime.datetime(2013, 1, 9, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 1, 10, 0, 0),
  datetime.datetime(2013, 1, 10, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 2, 27, 0, 0),
  datetime.datetime(2013, 2, 27, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 2, 28, 0, 0),
  datetime.datetime(2013, 2, 28, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 8, 0, 0),
  datetime.datetime(2013, 3, 8, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 9, 0, 0),
  datetime.datetime(2013, 3, 9, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 10, 0, 0),
  datetime.datetime(2013, 3, 10, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 28, 0, 0),
  datetime.datetime(2013, 3, 28, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 29, 0, 0),
  datetime.datetime(2013, 3, 29, 23, 59, 59, 999000)),
 (datetime.datetime(2013, 3, 30, 0, 0),
  datetime.datetim

In [10]:
from rl_env.env_data_loader import SmartMeterDataLoader

sm_dl_train = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_train,
    aggregate_load_df=aggregate_load_df_train
)

sm_dl_train.get_divided_segments_length()

162

In [11]:
sm_dl_train.divided_segments[7]

array([datetime.datetime(2013, 3, 10, 0, 0),
       datetime.datetime(2013, 3, 10, 23, 59, 59, 999000)], dtype=object)

In [12]:
# sample segment

sm_dl_train.get_aggregate_load_segment(13)

,timestamp,aggregate,datetime
11231,1363996805,335.000000,2013-03-23 00:00:05
11232,1363996865,317.206591,2013-03-23 00:01:05
11233,1363996925,293.301546,2013-03-23 00:02:05
11234,1363996985,276.229767,2013-03-23 00:03:05
11235,1363997045,276.501419,2013-03-23 00:04:05
...,...,...,...
12666,1364082905,173.758178,2013-03-23 23:55:05
12667,1364082965,172.392595,2013-03-23 23:56:05
12668,1364083025,174.270419,2013-03-23 23:57:05
12669,1364083085,171.201633,2013-03-23 23:58:05


In [13]:
# create dataloader for validation and test sets
sm_dl_validation = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_validation,
    aggregate_load_df=aggregate_load_df_validation
)

sm_dl_test = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_test,
    aggregate_load_df=aggregate_load_df_test
)

(Optional) Load the pre-trained H-network and related components

In [14]:

# from model.H_network.h_network import HNetwork

# h_network_datetime = datetime(2025, 7, 13)

# h_network_path = Path("model_trained", f"h_network_{h_network_datetime.strftime('%Y%m%d')}.pth")

# h_network = HNetwork(2, 44, 1)
# h_network.load_state_dict(torch.load(h_network_path))
# h_network.eval()

Create the environment

In [15]:
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
import sys
sys.path.append(str(Path('model', 'H_network')))
from model.H_network.h_network_rl_module import HNetworkRLModule
from model.H_network.h_network_arch import HNetworkType

h_network_rl_module = HNetworkRLModule(
    h_network_type=HNetworkType.H_NETWORK2,
    device=DEVICE
)

In [17]:
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env_hnetwork_loop import SmartMeterWorld

env_train = SmartMeterWorld(
    smart_meter_data_loader=sm_dl_train,
    h_network_rl_module=h_network_rl_module,
    # render_mode="human",
    render_mode=None,
)

In [18]:
HNetworkType.H_NETWORK2 is h_network_rl_module.h_network_type

True

In [19]:
h_network_rl_module.set_h_network(
    h_network_rl_module.initialize_h_network()
)
h_network_rl_module.initialize_h_network_training()

In [20]:
from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env_train)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

[2025-07-16 16:18:02:849] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-07-29 00:00:04'), Timestamp('2013-07-29 23:59:04'))}
[2025-07-16 16:18:02:853] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 693, 'datetime_range': (Timestamp('2013-01-10 00:00:02'), Timestamp('2013-01-10 11:32:02'))}
[2025-07-16 16:18:02:855] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-09-26 00:00:00'), Timestamp('2013-09-26 23:59:00'))}
[2025-07-16 16:18:02:857] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 693, 'datetime_range': (Timestamp('2013-01-10 00:00:02'), Timestamp('2013-01-10 11:32:02'))}
[2025-07-16 16:18:02:860] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-09-26 00:00:00'), Timestamp('2

/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


In [21]:
obs, info = env_train.reset()
obs

[2025-07-16 16:18:03:044] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-09-26 00:00:00'), Timestamp('2013-09-26 23:59:00'))}


{'aggregate_load': array([-0.4250123], dtype=float32),
 'battery_soc': array([0.], dtype=float32),
 'timestamp_features': array([-0.5 ,  0.  ,  0.25], dtype=float32)}

In [22]:
env_train.reset_render_window()

In [23]:
from stable_baselines3.common.callbacks import EveryNTimesteps, ConvertCallback
from typing import Any

class TrainHNetworkEveryNTimesteps(EveryNTimesteps):
    def __init__(self, n_steps: int, h_network_rl_module: HNetworkRLModule):
        super().__init__(n_steps=n_steps, callback=ConvertCallback(self._train))
        self.h_network_rl_module = h_network_rl_module

    def _train(self, _locals: dict[str, Any], _globals: dict[str, Any]) -> bool:
        
        print_log("Training H-network...")

        # Train the H-network
        self.h_network_rl_module.train()

        if self.h_network_rl_module.train_loss_list is None:
            print_log("No training loss data available. Skipping logging.")
            return True

        mean_loss = self.h_network_rl_module.train_loss_list[-1][0]
        std_loss = self.h_network_rl_module.train_loss_list[-1][1]
        print_log(f"(mean, std): ({mean_loss:.8f}, {std_loss:.8f})")

        return True

2025-07-16 16:18:03.297514: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 16:18:03.307043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752679083.317412 1339424 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752679083.320518 1339424 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752679083.329673 1339424 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [24]:
# initialize experiment folder to store related information/data for future analysis

rl_datetime = datetime.now()

experiment_folder = Path(
    "experiments", rl_datetime.strftime('%Y%m%d_%H%M%S')
)

if not experiment_folder.exists():
    experiment_folder.mkdir(parents=True)

In [25]:
# initialize a PPO agent
from stable_baselines3 import PPO

rl_datetime = datetime.now()
tensorboard_log_path = experiment_folder / "PPO"

rl_model = PPO(
    "MultiInputPolicy", 
    env_train, 
    verbose=2,
    tensorboard_log=tensorboard_log_path
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [26]:
rl_model.learn(
    total_timesteps=24 * 60 * 5 * 50,
    progress_bar=True,
    tb_log_name="PPO_SmartMeterWorld",
    callback=[TrainHNetworkEveryNTimesteps(n_steps=24 * 60 * 5, h_network_rl_module=h_network_rl_module)]
)

[2025-07-16 16:18:08:097] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-08-28 00:00:02'), Timestamp('2013-08-28 23:59:02'))}
Logging to experiments/20250716_161804/PPO/PPO_SmartMeterWorld_1


Output()

[2025-07-16 16:18:15:771] [SmartMeterWorld] Episode finished. Sum of rewards: 53.05696575434474. Mean of rewards: 
0.03687071977369336. Std of rewards: 0.02803334091201832

[2025-07-16 16:18:15:778] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 
1440, 'datetime_range': (Timestamp('2013-03-28 00:00:05'), Timestamp('2013-03-28 23:59:05'))}

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.44e+03 |
|    ep_rew_mean     | 53.1     |
| time/              |          |
|    fps             | 190      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------


...

[2025-07-16 16:53:24:649] Training H-network...

[2025-07-16 16:53:24:678] (mean, std): (-1.11149284, 0.11501208)

[2025-07-16 16:53:26:874] [SmartMeterWorld] Episode finished. Sum of rewards: -185.14684718567568. Mean of rewards:
-0.12866354912138678. Std of rewards: 1.3343275158492247

[2025-07-16 16:53:26:877] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 
1440, 'datetime_range': (Timestamp('2013-10-29 00:00:01'), Timestamp('2013-10-29 23:59:01'))}

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.44e+03     |
|    ep_rew_mean          | -1.08e+03    |
| time/                   |              |
|    fps                  | 170          |
|    iterations           | 176          |
|    time_elapsed         | 2119         |
|    total_timesteps      | 360448       |
| train/                  |              |
|    approx_kl            | 0.0019572652 |
|    clip_fraction        | 0.0145       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.735       |
|    explained_variance   | 0.872        |
|    learning_rate        | 0.0003       |
|    loss                 | 58.4         |
|    n_updates            | 1750         |
|    policy_gradient_loss | -0.000959    |
|    std                  | 0.5          |
|    value_loss           | 167          |
------------------------------------------


Create a validation environment

and put the policy into the validation env

In [41]:
import sys
sys.path.append(str(Path('rl_env')))
from rl_env.hrl_env_hnetwork_loop import SmartMeterWorld

env_valid = SmartMeterWorld(
    smart_meter_data_loader=sm_dl_validation,
    h_network_rl_module=h_network_rl_module,
    render_mode="human",
)

[2025-07-17 00:25:00:602] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=89, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 34266), raddr=('127.0.0.1', 50007)>


In [47]:
env_valid.reset_render_window()

In [43]:
rl_model.set_env(env_valid)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [48]:
env_valid_seed = 5

obs, info = env_valid.reset(env_valid_seed)
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env_valid.step(action)
        print_log(f"Step: {env_valid.episode.get_current_step()}, Action: {action}, Reward: {reward}")
        env_valid.render()

[2025-07-17 00:27:10:990] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 1440, 'datetime_range': (Timestamp('2013-09-11 00:00:02'), Timestamp('2013-09-11 23:59:02'))}
[2025-07-17 00:27:11:000] Step: 1, Action: [0.8147611], Reward: 0.17708887810905774
[2025-07-17 00:27:11:005] Step: 2, Action: [0.81093997], Reward: 0.1811874174773693
[2025-07-17 00:27:11:009] Step: 3, Action: [0.80468327], Reward: 0.18105786076188088
[2025-07-17 00:27:11:014] Step: 4, Action: [0.80429995], Reward: 0.18087816176017124
[2025-07-17 00:27:11:017] Step: 5, Action: [0.8104201], Reward: 0.18079597077171009
[2025-07-17 00:27:11:021] Step: 6, Action: [0.8093508], Reward: 0.18078525075713794
[2025-07-17 00:27:11:026] Step: 7, Action: [0.8172339], Reward: 0.18077042319377262
[2025-07-17 00:27:11:031] Step: 8, Action: [0.8111543], Reward: 0.18081671494046847
[2025-07-17 00:27:11:038] Step: 9, Action: [0.8107143], Reward: 0.18083522830605506
[2025-07-17 00:27:11:043] Step: 10, A

In [45]:
env_valid.episode.df

,timestamp,aggregate,datetime,grid_load,battery_soc,aggregate_std
98417,1386720004,274.000000,2013-12-11 00:00:04,4127.707075,0.0,-0.425049
98418,1386720064,274.857664,2013-12-11 00:01:04,4112.500095,0.008029,-0.422770
98419,1386720124,275.368756,2013-12-11 00:02:04,4101.837463,0.016024,-0.421411
98420,1386720184,228.711933,2013-12-11 00:03:04,4228.711933,0.023995,-0.545419
98421,1386720244,211.047353,2013-12-11 00:04:04,4211.047353,0.032329,-0.592369
...,...,...,...,...,...,...
99852,1386806104,348.279965,2013-12-11 23:55:04,2756.595385,0.499239,-0.227622
99853,1386806164,348.376078,2013-12-11 23:56:04,2742.894215,0.504256,-0.227367
99854,1386806224,348.769009,2013-12-11 23:57:04,2717.696249,0.509245,-0.226323
99855,1386806284,348.182031,2013-12-11 23:58:04,2730.595179,0.51418,-0.227883


In [49]:
# save the graph

env_valid.save_graph(
    {"fname": str(experiment_folder / f"graph_valid_index_{env_valid.selected_idx}.png"),
     "dpi": 300}
)

In [73]:
env_valid.close()

[2025-07-16 06:31:55:987] [SmartMeterWorld] Environment closed.


In [27]:
# save the model
rl_model_path = experiment_folder /"rl_model.zip"
rl_model.save(rl_model_path)

In [28]:
# save the H-network
h_network_path = experiment_folder / "h_network2.pth"
h_network_rl_module.save_h_network(h_network_path)

In [ ]:
h_network_rl_module.save_train_loss_list(experiment_folder / "train_loss_list.npy")

---

In [ ]:
# load the model & environment
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld
from stable_baselines3 import PPO
from model.H_network.h_network_arch import HNetworkType

env_test = SmartMeterWorld(
    sm_dl_test,
    h_model_type=HNetworkType.H_NETWORK,
    render_mode="human",
)

env_test.set_h_network(h_network)
env_test.set_h_network_stdscaler(h_network_stdscaler)

rl_model_path = Path("rl_model", "PPO", f"{datetime(2025,7,12,18,25,2).strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model_loaded = PPO.load(rl_model_path, env=env_test)

In [ ]:
obs, info = env_test.reset(43)

In [ ]:
info

In [ ]:
env_test.reset_render_window()

In [ ]:
env_test.episode.df

In [ ]:
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model_loaded.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env_test.step(action)
        print_log(f"Step: {env_test.episode.get_current_step()}, Action: {action}, Reward: {reward}, Info: {info}")
        env_test.render()

In [ ]:
env_test.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph_test.png"))
)

In [ ]:
env_test.close()